# Walk-Forward Validation for Insurance Strategies

This notebook demonstrates the walk-forward validation system for testing and comparing different insurance strategies.

## Overview

Walk-forward validation helps:
1. Test strategies across multiple time windows
2. Detect overfitting in optimization-based strategies
3. Measure strategy consistency and robustness
4. Rank strategies based on out-of-sample performance

## Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
parent_dir = Path.cwd().parent.parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Import our modules
from ergodic_insurance.config import Config
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.strategy_backtester import (
    NoInsuranceStrategy,
    ConservativeFixedStrategy,
    AggressiveFixedStrategy,
    OptimizedStaticStrategy,
    AdaptiveStrategy,
    StrategyBacktester
)
from ergodic_insurance.validation_metrics import (
    MetricCalculator,
    PerformanceTargets
)
from ergodic_insurance.walk_forward_validator import WalkForwardValidator

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Setup complete!")

## 1. Define Insurance Strategies

We'll test five different strategies with varying risk profiles:

In [ ]:
# Create strategies to test
strategies = [
    NoInsuranceStrategy(),
    ConservativeFixedStrategy(
        primary_limit=5000000,
        excess_limit=20000000,
        higher_limit=25000000,
        deductible=50000
    ),
    AggressiveFixedStrategy(
        primary_limit=2000000,
        excess_limit=5000000,
        deductible=250000
    ),
    OptimizedStaticStrategy(
        target_roe=0.15,
        max_ruin_prob=0.01
    ),
    AdaptiveStrategy(
        base_deductible=100000,
        base_primary=3000000,
        base_excess=10000000,
        adaptation_window=3,
        adjustment_factor=0.2
    )
]

# Display strategy descriptions
for strategy in strategies:
    print(f"- {strategy.name}: {strategy.get_description()}")

## 2. Set Performance Targets (Optional)

Define minimum acceptable performance criteria:

In [ ]:
# Define performance targets
performance_targets = PerformanceTargets(
    min_roe=0.10,  # Minimum 10% ROE
    max_ruin_probability=0.05,  # Maximum 5% ruin probability
    min_sharpe_ratio=0.8,  # Minimum Sharpe ratio
    max_drawdown=0.30,  # Maximum 30% drawdown
    min_growth_rate=0.05  # Minimum 5% growth rate
)

print("Performance Targets Set:")
print(f"  Min ROE: {performance_targets.min_roe:.1%}")
print(f"  Max Ruin Probability: {performance_targets.max_ruin_probability:.1%}")
print(f"  Min Sharpe Ratio: {performance_targets.min_sharpe_ratio:.2f}")
print(f"  Max Drawdown: {performance_targets.max_drawdown:.1%}")
print(f"  Min Growth Rate: {performance_targets.min_growth_rate:.1%}")

## 3. Configure Walk-Forward Validation

Set up the validation with 3-year rolling windows:

In [ ]:
# Create validator with 3-year windows
validator = WalkForwardValidator(
    window_size=3,  # 3-year windows
    step_size=1,    # 1-year step between windows
    test_ratio=0.3, # 30% of window for testing (approximately 1 year)
    performance_targets=performance_targets
)

# Show window configuration
total_years = 10
windows = validator.generate_windows(total_years)

print(f"Validation Configuration:")
print(f"  Total years: {total_years}")
print(f"  Number of windows: {len(windows)}")
print(f"\nFirst 3 windows:")
for window in windows[:3]:
    print(f"  {window}")

## 4. Run Walk-Forward Validation

This will test each strategy across all windows (may take a few minutes):

In [ ]:
# Configure manufacturer and simulation settings
from ergodic_insurance.config import ManufacturerConfig

# Create manufacturer configuration
manufacturer_config = ManufacturerConfig(
    initial_assets=10000000,
    asset_turnover_ratio=1.0,
    base_operating_margin=0.08,
    tax_rate=0.25,
    retention_ratio=0.95  # Retain 95% of earnings (5% capex)
)

# Create manufacturer instance
manufacturer = WidgetManufacturer(manufacturer_config)

# Run validation (simplified for demo - use more simulations in production)
print("Running walk-forward validation...")
print("(This may take a few minutes)\n")

validation_result = validator.validate_strategies(
    strategies=strategies,
    n_years=total_years,
    n_simulations=100,  # Use 1000+ for production
    manufacturer=manufacturer
)

print(f"\n✓ Validation complete!")
print(f"Best strategy: {validation_result.best_strategy}")

## 5. Analyze Results

### Strategy Rankings

In [ ]:
# Display strategy rankings
print("Strategy Rankings (by composite score):")
print("="*60)

if not validation_result.strategy_rankings.empty:
    rankings_display = validation_result.strategy_rankings[[
        'strategy', 'avg_roe', 'avg_ruin_prob', 'avg_sharpe',
        'overfitting_score', 'consistency_score', 'composite_score'
    ]].round(4)
    
    # Format percentages
    rankings_display['avg_roe'] = rankings_display['avg_roe'].apply(lambda x: f"{x:.2%}")
    rankings_display['avg_ruin_prob'] = rankings_display['avg_ruin_prob'].apply(lambda x: f"{x:.2%}")
    
    print(rankings_display.to_string(index=False))
else:
    print("No rankings available")

### Overfitting Analysis

In [ ]:
# Analyze overfitting
print("Overfitting Analysis:")
print("="*40)

for strategy_name, score in validation_result.overfitting_analysis.items():
    if score < 0.2:
        status = "✓ Low (Good)"
        color = 'green'
    elif score < 0.4:
        status = "⚠ Moderate"
        color = 'orange'
    else:
        status = "✗ High (Poor)"
        color = 'red'
    
    print(f"{strategy_name:25} {score:.3f} - {status}")

# Visualize overfitting scores
if validation_result.overfitting_analysis:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    strategies_list = list(validation_result.overfitting_analysis.keys())
    scores = list(validation_result.overfitting_analysis.values())
    
    bars = ax.bar(strategies_list, scores)
    
    # Color based on severity
    for bar, score in zip(bars, scores):
        if score < 0.2:
            bar.set_color('green')
        elif score < 0.4:
            bar.set_color('orange')
        else:
            bar.set_color('red')
    
    ax.set_title('Overfitting Scores by Strategy', fontsize=14, fontweight='bold')
    ax.set_xlabel('Strategy')
    ax.set_ylabel('Overfitting Score')
    ax.axhline(y=0.2, color='orange', linestyle='--', alpha=0.5, label='Moderate threshold')
    ax.axhline(y=0.4, color='red', linestyle='--', alpha=0.5, label='High threshold')
    ax.legend()
    ax.set_ylim(0, max(scores) * 1.2 if scores else 1)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

### Performance Across Windows

In [ ]:
# Plot performance metrics across windows
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Collect data
strategy_names = list(validation_result.window_results[0].strategy_performances.keys())
windows_idx = list(range(len(validation_result.window_results)))

metrics_data = {
    'Out-Sample ROE': {s: [] for s in strategy_names},
    'Ruin Probability': {s: [] for s in strategy_names},
    'Sharpe Ratio': {s: [] for s in strategy_names},
    'Growth Rate': {s: [] for s in strategy_names}
}

for window_result in validation_result.window_results:
    for strategy in strategy_names:
        if strategy in window_result.strategy_performances:
            perf = window_result.strategy_performances[strategy]
            if perf.out_sample_metrics:
                metrics_data['Out-Sample ROE'][strategy].append(perf.out_sample_metrics.roe)
                metrics_data['Ruin Probability'][strategy].append(perf.out_sample_metrics.ruin_probability)
                metrics_data['Sharpe Ratio'][strategy].append(perf.out_sample_metrics.sharpe_ratio)
                metrics_data['Growth Rate'][strategy].append(perf.out_sample_metrics.growth_rate)

# Plot each metric
for ax, (metric_name, metric_values) in zip(axes.flat, metrics_data.items()):
    for strategy in strategy_names:
        if metric_values[strategy]:
            ax.plot(windows_idx[:len(metric_values[strategy])], 
                   metric_values[strategy], 
                   marker='o', 
                   label=strategy,
                   linewidth=2)
    
    ax.set_title(metric_name, fontweight='bold')
    ax.set_xlabel('Window')
    ax.set_ylabel(metric_name)
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.suptitle('Strategy Performance Across Validation Windows', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Strategy Consistency

In [ ]:
# Analyze consistency
print("Strategy Consistency Scores:")
print("="*40)

consistency_data = []
for strategy_name, score in validation_result.consistency_scores.items():
    if score > 0.8:
        status = "✓ High"
    elif score > 0.6:
        status = "⚠ Moderate"
    else:
        status = "✗ Low"
    
    print(f"{strategy_name:25} {score:.3f} - {status}")
    consistency_data.append({'Strategy': strategy_name, 'Consistency': score})

# Visualize consistency
if consistency_data:
    df_consistency = pd.DataFrame(consistency_data)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(df_consistency['Strategy'], df_consistency['Consistency'])
    
    # Color based on level
    for bar, score in zip(bars, df_consistency['Consistency']):
        if score > 0.8:
            bar.set_color('green')
        elif score > 0.6:
            bar.set_color('orange')
        else:
            bar.set_color('red')
    
    ax.set_xlabel('Consistency Score')
    ax.set_title('Strategy Consistency Across Windows', fontsize=14, fontweight='bold')
    ax.set_xlim(0, 1)
    ax.axvline(x=0.6, color='orange', linestyle='--', alpha=0.5)
    ax.axvline(x=0.8, color='green', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

## 6. Performance Target Evaluation

In [ ]:
# Evaluate each strategy against performance targets
print("Performance Target Evaluation:")
print("="*60)

for window_result in validation_result.window_results[-1:]:  # Use last window
    for strategy_name, performance in window_result.strategy_performances.items():
        if performance.out_sample_metrics:
            meets_targets, failures = performance_targets.evaluate(performance.out_sample_metrics)
            
            print(f"\n{strategy_name}:")
            if meets_targets:
                print("  ✓ Meets all targets")
            else:
                print("  ✗ Fails targets:")
                for failure in failures:
                    print(f"    - {failure}")

## 7. Generate Validation Reports

In [ ]:
# Generate comprehensive reports
import tempfile
from pathlib import Path

# Create temporary directory for reports
with tempfile.TemporaryDirectory() as temp_dir:
    report_dir = Path(temp_dir)
    
    print("Generating validation reports...")
    report_files = validator.generate_report(
        validation_result=validation_result,
        output_dir=str(report_dir),
        include_visualizations=True
    )
    
    print("\nGenerated files:")
    for file_type, file_path in report_files.items():
        if isinstance(file_path, Path):
            print(f"  - {file_type}: {file_path.name}")
    
    # Read and display markdown summary
    if 'markdown' in report_files:
        md_content = report_files['markdown'].read_text()
        print("\n" + "="*60)
        print("MARKDOWN SUMMARY (first 1000 chars):")
        print("="*60)
        print(md_content[:1000] + "..." if len(md_content) > 1000 else md_content)

## 8. Key Insights and Recommendations

Based on the walk-forward validation results:

In [ ]:
print("KEY INSIGHTS:")
print("="*60)

# Best strategy
print(f"\n1. RECOMMENDED STRATEGY: {validation_result.best_strategy}")

# Check for overfitting
overfit_strategies = [s for s, score in validation_result.overfitting_analysis.items() if score > 0.4]
if overfit_strategies:
    print(f"\n2. OVERFITTING WARNING:")
    for s in overfit_strategies:
        print(f"   - {s} shows significant overfitting")
else:
    print(f"\n2. OVERFITTING: No strategies show significant overfitting")

# Consistency leaders
if validation_result.consistency_scores:
    most_consistent = max(validation_result.consistency_scores.items(), key=lambda x: x[1])
    print(f"\n3. MOST CONSISTENT: {most_consistent[0]} (score: {most_consistent[1]:.3f})")

# Performance vs targets
print(f"\n4. PERFORMANCE TARGETS:")
if not validation_result.strategy_rankings.empty:
    high_performers = validation_result.strategy_rankings[
        validation_result.strategy_rankings['avg_roe'] >= performance_targets.min_roe
    ]['strategy'].tolist()
    
    if high_performers:
        print(f"   Strategies meeting ROE target: {', '.join(high_performers)}")
    else:
        print(f"   No strategies meet the minimum ROE target")

# Final recommendation
print(f"\n5. FINAL RECOMMENDATION:")
print(f"   Deploy {validation_result.best_strategy} strategy based on:")
print(f"   - Strong out-of-sample performance")
print(f"   - Acceptable overfitting risk")
print(f"   - Good consistency across market conditions")

## Summary

This walk-forward validation system provides:

1. **Robust Testing**: Strategies tested across multiple time windows
2. **Overfitting Detection**: Identifies strategies that perform well in-sample but poorly out-of-sample
3. **Consistency Measurement**: Evaluates strategy stability across different periods
4. **Performance Tracking**: Comprehensive metrics for informed decision-making
5. **Actionable Insights**: Clear recommendations based on empirical evidence

The system helps ensure that selected insurance strategies will perform well in real-world conditions, not just in backtests.